### MQTT Scriptlets

Short segments of setup/loop python will be delivered through MQTT and then executed 
in a system that is managing its external work using the async system, so that it can 
restart or handle incoming or outgoing MQTT messages.

This will be the ultimate in microcontroller configurability and remote operations based 
on a standard install.

The async MQTT project is at https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as

Command line code for handling MQTT
* mosquitto_sub  -v -h mqtt.local -t "scriptlet1/#"
* mosquitto_pub  -h mqtt.local -t "scriptlet1/code" -m "gothere=999"

In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [278]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/


Sent 28 lines (846 bytes) to lib/uasyncio/synchro.py.


In [279]:
# This requires you to checkout https://github.com/peterhinch/micropython-async
%sendtofile --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --source /home/julian/extrepositories/micropython-async/aswitch.py

Sent 188 lines (7484 bytes) to aswitch.py.


In [280]:
# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

Sent 638 lines (23042 bytes) to mqtt_as.py.


In [ ]:
%sendtofile --source sevensegmentdisplay.py

In [281]:
# Set up the wifi connection
%sendtofile config.py

from mqtt_as import config, ESP32

# Include any cross-project settings.

if ESP32:
    config['ssid'] = 'DoESLiverpool'  # EDIT if you're using ESP32
    config['wifi_pw'] = 'decafbad00'
    config['mqttchannel'] = "scriptlet1"

Sent 8 lines (228 bytes) to config.py.


In [85]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [282]:
%sendtofile scriptlet.py

@asyn.cancellable
async def setup():
    pass

@asyn.cancellable
async def loop():
    pass

Sent 7 lines (91 bytes) to scriptlet.py.


In [284]:
#%sendtofile main.py

from mqtt_as import MQTTClient
from config import config
import uasyncio as asyncio
import asyn
import os

SERVER = "10.0.29.167"  # mqtt.local
mqttchannel = config["mqttchannel"]
client, aloop = None, None

# should we carry on with previous copy?
# We'd then have to call it
#execfile("scriptlet.py")  

# This is the MQTT batching code with headers 001/006]
slsegment = -1
slsegmentN = -1
def appendtoscriptletfile(msg):
    global slsegment, slsegmentN
    if len(msg) > 8 and msg[7] == ord("]"):
        if slsegment == -1:
            if int(msg[:3]) != 1:
                print("Bad", msg[:8])
                client.publish(mqttchannel+'/bad', msg[:8])
                return False
            slsegmentN = int(msg[4:7])
            slsegment = 1
            fin = open("scriptlet_NEW.py", "wb")
            fin.write(msg[8:])
            fin.close()
        elif int(msg[:3]) != slsegment+1 or slsegmentN != int(msg[4:7]):
            slsegment = -1
            print("Bad", msg[:8])
            client.publish(mqttchannel+'/bad', msg[:8])
            return False
        else:
            slsegment += 1
            fin = open("scriptlet_NEW.py", "ab")
            fin.write(msg[8:])
            fin.close()
            
        if slsegment != slsegmentN:
            return False
            
    else:  # Non-chunked case
        fin = open("scriptlet_NEW.py", "a")
        fin.write(msg)
        fin.close()
    slsegment = -1  # ready for start of next batch
    return True


@asyn.cancellable
async def setloop():
    try:
        await asyn.Cancellable(setup)()
        while True:
            await asyn.sleep(0.1)
            await asyn.Cancellable(loop)()
    except Exception as e:
        await client.publish(mqttchannel+'/rec', 'error '+str(e))

        
async def acallback(topic, msg):
    print((topic, msg))
    await client.publish(mqttchannel+'/rec', b'received '+msg[:8])

    if not appendtoscriptletfile(msg):
        return
    os.rename("scriptlet_NEW.py", "scriptlet.py")
    await client.publish(mqttchannel+'/rec', 'abouttocancel_all')
    await asyn.Cancellable.cancel_all()
    await client.publish(mqttchannel+'/rec', 'abouttoexec')
    try:
        execfile("scriptlet.py")
    except Exception as e:
        print(e)
        await client.publish(mqttchannel+'/rec', str(e))
        return
    
    await client.publish(mqttchannel+'/rec', 'abouttorun')
    aloop.create_task(asyn.Cancellable(setloop)())

# The callback has to be a normal function, but we can create an async task from within it using aloop
def callback(topic, msg):
    aloop.create_task(acallback(topic, msg))
    
async def conn_han(client):
    print("subscribing", (mqttchannel+'/code'))
    await client.subscribe(mqttchannel+'/code', 1)
    
async def main(client):
    await client.connect()
    n = 0
    while True:
        await asyncio.sleep(5)
        await client.publish(mqttchannel+'/beat', '{}'.format(n), qos = 1)
        n += 1

config['subs_cb'] = callback
config['connect_coro'] = conn_han
config['server'] = SERVER

MQTTClient.DEBUG = False  # Optional: print diagnostic messages
client = MQTTClient(config)
aloop = asyncio.get_event_loop()
try:
    aloop.run_until_complete(main(client))
finally:
    client.close()  # Prevent LmacRxBlk:1 errors

Sent 110 lines (3280 bytes) to main.py.


In [ ]:
%sendtofile --source sevensegmentdisplay.py

In [150]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [146]:
import os
print(os.listdir())

['boot.py', 'OLED_driver.py', 'lib', 'utils.py', 'main.py', 'device_detection.py', 'SI7021_funcs.py', 'TSL561_funcs.py', 'VL53L0X_funcs.py', 'VL6180_funcs.py', 'SDOF_funcs.py', 'SHT31D_funcs.py', 'BME280_funcs.py', 'MLX90621_funcs.py', 'BNO055serial_funcs.py', 'LOG', '001.TXT', 'BNO055_funcs.py']


Sent 76 lines (2196 bytes) to sevensegmentdisplay.py.


In [273]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [274]:
from sevensegmentdisplay import encodeledstring, writeledstringautoscroll

In [275]:
encodeledstring("stale -- days old")
while 1:
    writeledstringautoscroll()


......................................................................................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
  File "sevensegmentdisplay.py", line 74, in writeledstringautoscroll
  File "sevensegmentdisplay.py", line 13, in swrite
KeyboardInterrupt: 


In [276]:
def phrases():
    yield 1, "fresh"
    for h in range(1, 4):
        yield 1, "fresh %d hours" % h
    yield 1, "stale"
    for h in range(4, 24):
        yield 1, "stale %d hours" % h
    for d in range(1, 100):
        yield 24, "bad %d days old" % d
    
print(list(phrases()))

[(1, 'fresh'), (1, 'fresh 1 hours'), (1, 'fresh 2 hours'), (1, 'fresh 3 hours'), (1, 'stale'), (1, 'stale 4 hours'), (1, 'stale 5 hours'), (1, 'stale 6 hours'), (1, 'stale 7 hours'), (1, 'stale 8 hours'), (1, 'stale 9 hours'), (1, 'stale 10 hours'), (1, 'stale 11 hours'), (1, 'stale 12 hours'), (1, 'stale 13 hours'), (1, 'stale 14 hours'), (1, 'stale 15 hours'), (1, 'stale 16 hours'), (1, 'stale 17 hours'), (1, 'stale 18 hours'), (1, 'stale 19 hours'), (1, 'stale 20 hours'), (1, 'stale 21 hours'), (1, 'stale 22 hours'), (1, 'stale 23 hours'), (24, 'bad 1 days old'), (24, 'bad 2 days old'), (24, 'bad 3 days old'), (24, 'bad 4 days old'), (24, 'bad 5 days old'), (24, 'bad 6 days old'), (24, 'bad 7 days old'), (24, 'bad 8 days old'), (24, 'bad 9 days old'), (24, 'bad 10 days old'), (24, 'bad 11 days old'), (24, 'bad 12 days old'), (24, 'bad 13 days old'), (24, 'bad 14 days old'), (24, 'bad 15 days old'), (24, 'bad 16 days old'), (24, 'bad 17 days old'), (24, 'bad 18 days old'), (24, 'bad 